In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

# Model construction utilities below adapted from
# https://www.tensorflow.org/versions/r0.8/tutorials/mnist/pros/index.html#deep-mnist-for-experts
def weight_variable(shape):
    initial = tf.random.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def shuffle_aligned_list(data):
    """Shuffle arrays in a list by shuffling each array identically."""
    num = data[0].shape[0]
    p = np.random.permutation(num)
    return [d[p] for d in data]


def batch_generator(data, batch_size, shuffle=True):
    """Generate batches of data.
    
    Given a list of array-like objects, generate batches of a given
    size by yielding a list of array-like objects corresponding to the
    same slice of each input.
    """
    if shuffle:
        data = shuffle_aligned_list(data)

    batch_count = 0
    while True:
        if batch_count * batch_size + batch_size >= len(data[0]):
            batch_count = 0

            if shuffle:
                data = shuffle_aligned_list(data)

        start = batch_count * batch_size
        end = start + batch_size
        batch_count += 1
        yield [d[start:end] for d in data]


def imshow_grid(images, shape=[2, 8]):
    """Plot images in a grid of a given shape."""
    fig = plt.figure(1)
    grid = ImageGrid(fig, 111, nrows_ncols=shape, axes_pad=0.05)

    size = shape[0] * shape[1]
    for i in range(size):
        grid[i].axis('off')
        grid[i].imshow(images[i])  # The AxesGrid object work as a list of axes.

    plt.show()


def plot_embedding(X, y, d, title=None):
    """Plot an embedding X with the class label y colored by the domain d."""
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    # Plot colors numbers
    plt.figure(figsize=(10,10))
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        # plot colored number
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.bwr(d[i] / 1.),
                 fontdict={'weight': 'bold', 'size': 9})

    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import pickle as pkl
from sklearn.datasets import make_moons, make_blobs
from sklearn.decomposition import PCA

from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0,'/content/drive/MyDrive/CrossDomain_FakeReviewDetection/Datasets')

from flip_gradient import flip_gradient

Mounted at /content/drive


In [ ]:
!pip install tensorflow_text

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from google.colab import drive
df = pd.read_excel("/content/drive/MyDrive/CrossDomain_FakeReviewDetection/Datasets/fake reviews dataset___ includes computer generated ones.xlsx")
df.head()

In [ ]:
print(df.value_counts(['category','label']))
df['spam']=df['label'].apply(lambda x: 1 if x=='CG' else 0)
gk = df.groupby('category')

category                      label
Kindle_Store_5                CG       2365
                              OR       2365
Books_5                       OR       2185
                              CG       2185
Pet_Supplies_5                CG       2127
                              OR       2127
Home_and_Kitchen_5            CG       2028
                              OR       2028
Electronics_5                 CG       1994
                              OR       1994
Sports_and_Outdoors_5         CG       1973
                              OR       1973
Tools_and_Home_Improvement_5  OR       1929
                              CG       1929
Clothing_Shoes_and_Jewelry_5  OR       1924
                              CG       1924
Toys_and_Games_5              CG       1897
                              OR       1897
Movies_and_TV_5               OR       1794
                              CG       1794
dtype: int64


In [ ]:
df1 = gk.get_group('Kindle_Store_5')
df2 = gk.get_group('Books_5')
df3 = gk.get_group('Pet_Supplies_5')
# df4 = gk.get_group('')

In [ ]:
!pip install transformers 

import torch
from transformers import AutoTokenizer, AutoModel

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

In [ ]:

tokenized_train = tokenizer(df1["text_"].values.tolist(), padding = True, truncation = True, return_tensors="pt")
tokenized_val = tokenizer(df2["text_"].values.tolist() , padding = True, truncation = True,  return_tensors="pt")

print(tokenized_train.keys())

#move on device (GPU)
tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
tokenized_val = {k:torch.tensor(v).to(device) for k,v in tokenized_val.items()}

dict_keys(['input_ids', 'attention_mask'])


<ipython-input-7-c6774dd54581>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
<ipython-input-7-c6774dd54581>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_val = {k:torch.tensor(v).to(device) for k,v in tokenized_val.items()}


In [ ]:
batch_size = 8

# tokenize the training data
train_inputs = [df1["text_"].values[i:i+batch_size].tolist() for i in range(0, len(df1), batch_size)]
tokenized_train = [tokenizer(batch, padding=True, truncation=True, return_tensors="pt") for batch in train_inputs]

# tokenize the validation data
val_inputs = [df2["text_"].values[i:i+batch_size].tolist() for i in range(0, len(df2), batch_size)]
tokenized_val = [tokenizer(batch, padding=True, truncation=True, return_tensors="pt") for batch in val_inputs]

# move to device
tokenized_train = [{k: v.to(device) for k, v in batch.items()} for batch in tokenized_train]
tokenized_val = [{k: v.to(device) for k, v in batch.items()} for batch in tokenized_val]

xs = []
xt = []

with torch.no_grad():
    for batch in tokenized_train:
        hidden_train = model(**batch)
        cls_train = hidden_train.last_hidden_state[:, 0, :]
        xs.append(cls_train.to("cpu").numpy())
    for batch in tokenized_val:
        hidden_val = model(**batch)
        cls_val = hidden_val.last_hidden_state[:, 0, :]
        xt.append(cls_val.to("cpu").numpy())


In [ ]:
import numpy
Xs = numpy.vstack( xs )
Xt = numpy.vstack( xt)

In [ ]:
from sklearn.model_selection import train_test_split

ys = df1['spam'].to_numpy()
yt = df2['spam'].to_numpy()

print(Xs.shape, ys.shape)
print(Xt.shape, yt.shape)

Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)
Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, yt, test_size=0.2, random_state=42)


(4730, 768) (4730,)
(4370, 768) (4370,)


In [ ]:
batch_size = 16
FF=[];
def build_model(shallow_domain_classifier=True):
    X = tf.compat.v1.placeholder(tf.float32, [None, 768], name='X') # Input data
    Y_ind = tf.compat.v1.placeholder(tf.int32, [None], name='Y_ind')  # Class index
    D_ind = tf.compat.v1.placeholder(tf.int32, [None], name='D_ind')  # Domain index
    train = tf.compat.v1.placeholder(tf.bool, [], name='train')       # Switch for routing data to class predictor
    l = tf.compat.v1.placeholder(tf.float32, [], name='l')        # Gradient reversal scaler

    Y = tf.one_hot(Y_ind, 2)
    D = tf.one_hot(D_ind, 2)

    # Feature extractor - single layer
    W0 = weight_variable([768, 128])
    b0 = bias_variable([128])
    F = tf.nn.relu(tf.matmul(X, W0) + b0, name='feature')
    FF=F;
    # Label predictor - single layer
    f = tf.cond(train, lambda: tf.slice(F, [0, 0], [batch_size // 2, -1]), lambda: F)
    y = tf.cond(train, lambda: tf.slice(Y, [0, 0], [batch_size // 2, -1]), lambda: Y)
    

    W1 = weight_variable([128, 2])
    b1 = bias_variable([2])
    p_logit = tf.matmul(f, W1) + b1
    p = tf.nn.softmax(p_logit)
    p_loss = tf.nn.softmax_cross_entropy_with_logits(logits=p_logit, labels=y)

    # Domain predictor - shallow
    f_ = flip_gradient(F, l)

    if shallow_domain_classifier:
        W2 = weight_variable([128, 2])
        b2 = bias_variable([2])
        d_logit = tf.matmul(f_, W2) + b2
        d = tf.nn.softmax(d_logit)
        d_loss = tf.nn.softmax_cross_entropy_with_logits(logits=d_logit, labels=D)

    else:
        W2 = weight_variable([128, 128])
        b2 = bias_variable([128])
        h2 = tf.nn.relu(tf.matmul(f_, W2) + b2)

        W3 = weight_variable([128, 2])
        b3 = bias_variable([2])
        d_logit = tf.matmul(h2, W3) + b3
        d = tf.nn.softmax(d_logit)
        d_loss = tf.nn.softmax_cross_entropy_with_logits(logits=d_logit, labels=D)


    # Optimization
    pred_loss = tf.reduce_sum(p_loss, name='pred_loss')
    domain_loss = tf.reduce_sum(d_loss, name='domain_loss')
    total_loss = tf.add(pred_loss, domain_loss, name='total_loss')

    pred_train_op = tf.compat.v1.train.AdamOptimizer().minimize(pred_loss, name='pred_train_op')
    domain_train_op = tf.compat.v1.train.AdamOptimizer().minimize(domain_loss, name='domain_train_op')
    dann_train_op = tf.compat.v1.train.AdamOptimizer().minimize(total_loss, name='dann_train_op')

    # Evaluation
    p_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(p, 1)), tf.float32), name='p_acc')
    d_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(D, 1), tf.argmax(d, 1)), tf.float32), name='d_acc')

In [ ]:
tf.compat.v1.disable_eager_execution()
build_model()
sess = tf.compat.v1.InteractiveSession()
results=[[{} for x in range(6)] for y in range(6)]
# ix,iy=0,0
def train_and_evaluate(sess, train_op_name, train_loss_name,ix=0,iy=0, grad_scale=None, num_batches=200, verbose=True):
    # Create batch builders
    S_batches = batch_generator([Xs_train, ys_train], batch_size // 2)
    T_batches = batch_generator([Xt_train, yt_train], batch_size // 2)
    
    # Get output tensors and train op
    d_acc = sess.graph.get_tensor_by_name('d_acc:0')
    p_acc = sess.graph.get_tensor_by_name('p_acc:0')
    train_loss = sess.graph.get_tensor_by_name(train_loss_name + ':0')
    train_op = sess.graph.get_operation_by_name(train_op_name)
    
    sess.run(tf.compat.v1.global_variables_initializer())
    for i in range(num_batches):

        # If no grad_scale, use a schedule
        if grad_scale is None:
            p = float(i) / num_batches
            lp = 2. / (1. + np.exp(-10. * p)) - 1
        else:
            lp = grad_scale

        X0, y0 = next(S_batches)
        X1, y1 = next(T_batches)
        Xb = np.vstack([X0, X1])
        yb = np.hstack([y0, y1])
        D_labels = np.hstack([np.zeros(batch_size // 2, dtype=np.int32),
                              np.ones(batch_size // 2, dtype=np.int32)])

        _, loss, da, pa = sess.run([train_op, train_loss, d_acc, p_acc],
                                   feed_dict={'X:0': Xb, 'Y_ind:0': yb, 'D_ind:0': D_labels,
                                              'train:0': True, 'l:0': lp})

        if verbose and i % 200 == 0:
            print('loss: {}, domain accuracy: {}, class accuracy: {}'.format(loss, da, pa))

            
    # Get final accuracies on whole dataset
    das, pas = sess.run([d_acc, p_acc], feed_dict={'X:0': Xs_test, 'Y_ind:0': ys_test, 
                            'D_ind:0': np.zeros(Xs_test.shape[0], dtype=np.int32), 'train:0': False, 'l:0': 1.0})
    dat, pat = sess.run([d_acc, p_acc], feed_dict={'X:0': Xt_test, 'Y_ind:0': yt_test,
                            'D_ind:0': np.ones(Xt_test.shape[0], dtype=np.int32), 'train:0': False, 'l:0': 1.0})

    print('Source domain: ', das)
    print('Source class: ', pas)
    print('Target domain: ', dat)
    print('Target class: ', pat)
    results[ix][iy]['Source Domain']=das
    results[ix][iy]['Source Class']=pas
    results[ix][iy]['Target Domain']=dat
    results[ix][iy]['Target Class']=pat
    # extract_and_plot_pca_feats(sess)


def extract_and_plot_pca_feats(sess, feat_tensor_name='feature'):
    F = sess.graph.get_tensor_by_name(feat_tensor_name + ':0')
    emb_s = sess.run(F, feed_dict={'X:0': Xs})
    emb_t = sess.run(F, feed_dict={'X:0': Xt})
    emb_all = np.vstack([emb_s, emb_t])

    pca = PCA(n_components=2)
    pca_emb = pca.fit_transform(emb_all)

    num = pca_emb.shape[0] // 2
    plt.scatter(pca_emb[:num,0], pca_emb[:num,1], c=ys, cmap='coolwarm', alpha=0.4)
    plt.scatter(pca_emb[num:,0], pca_emb[num:,1], c=yt, cmap='cool', alpha=0.4)
    plt.show()



In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None 
import numpy as np
import re
import nltk

from gensim.models import word2vec

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
%matplotlib inline
def tsne_plot(model):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []
    
    domains=['Kindle_Store_5','Books_5','Pet_Supplies_5','Home_and_Kitchen_5','Sports_and_Outdoors_5','Electronics_5']

    for word in range(6):
        tokens.append((domains[word]))
        labels.append((model[word]))
    
    tsne_model = TSNE(perplexity=40, n_components=2, init='pca', n_iter=2500, random_state=23)
    new_values = tsne_model.fit_transform(tokens)

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])
        
    plt.figure(figsize=(16, 16)) 
    for i in range(len(x)):
        plt.scatter(x[i],y[i])
        plt.annotate(labels[i],
                     xy=(x[i], y[i]),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')
    plt.show()
# tsne_plot(df)

In [ ]:
# Kindle_Store_5                CG       2365
#                               OR       2365
# Books_5                       OR       2185
#                               CG       2185
# Pet_Supplies_5                CG       2127
#                               OR       2127
# Home_and_Kitchen_5            CG       2028
#                               OR       2028
# Electronics_5                 CG       1994
#                               OR       1994
# Sports_and_Outdoors_5         CG       1973
#                               OR       1973
DF=[]
DF.append(gk.get_group('Kindle_Store_5'))
DF.append(gk.get_group('Books_5'))
DF.append(gk.get_group('Pet_Supplies_5'))
DF.append(gk.get_group('Home_and_Kitchen_5'))
DF.append(gk.get_group('Electronics_5'))
DF.append(gk.get_group('Sports_and_Outdoors_5'))

# !pip install transformers 




# import torch
# from transformers import AutoTokenizer, AutoModel
def data_loader(x=0,y=0):
  ix=x;
  iy=y;
  # print(x,y)
  
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
  model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)






  tokenized_train = tokenizer(DF[x]["text_"].values.tolist(), padding = True, truncation = True, return_tensors="pt")
  tokenized_val = tokenizer(DF[y]["text_"].values.tolist() , padding = True, truncation = True,  return_tensors="pt")

  # print(tokenized_train.keys())

  #move on device (GPU)
  tokenized_train = {k:torch.tensor(v).to(device) for k,v in tokenized_train.items()}
  tokenized_val = {k:torch.tensor(v).to(device) for k,v in tokenized_val.items()}


  batch_size = 8

  # tokenize the training data
  # print(DF[x])
  train_inputs = [DF[x]["text_"].values[i:i+batch_size].tolist() for i in range(0, len(DF[x]), batch_size)]
  tokenized_train = [tokenizer(batch, padding=True, truncation=True, return_tensors="pt") for batch in train_inputs]

  # tokenize the validation data
  val_inputs = [DF[y]["text_"].values[i:i+batch_size].tolist() for i in range(0, len(DF[y]), batch_size)]
  tokenized_val = [tokenizer(batch, padding=True, truncation=True, return_tensors="pt") for batch in val_inputs]

  # move to device
  tokenized_train = [{k: v.to(device) for k, v in batch.items()} for batch in tokenized_train]
  tokenized_val = [{k: v.to(device) for k, v in batch.items()} for batch in tokenized_val]

  xs = []
  xt = []

  with torch.no_grad():
      for batch in tokenized_train:
          hidden_train = model(**batch)
          cls_train = hidden_train.last_hidden_state[:, 0, :]
          xs.append(cls_train.to("cpu").numpy())
      for batch in tokenized_val:
          hidden_val = model(**batch)
          cls_val = hidden_val.last_hidden_state[:, 0, :]
          xt.append(cls_val.to("cpu").numpy())




  import numpy
  Xs = numpy.vstack( xs )
  Xt = numpy.vstack( xt)





  from sklearn.model_selection import train_test_split

  ys = DF[x]['spam'].to_numpy()
  yt = DF[y]['spam'].to_numpy()

  # print(Xs.shape, ys.shape)
  # print(Xt.shape, yt.shape)

  Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, test_size=0.2, random_state=42)
  Xt_train, Xt_test, yt_train, yt_test = train_test_split(Xt, yt, test_size=0.2, random_state=42)
  
  domains=['Kindle_Store_5','Books_5','Pet_Supplies_5','Home_and_Kitchen_5','Sports_and_Outdoors_5','Electronics_5']
  print( domains[x] +" vs "+domains[y] )
  batch_size=16
# for i in range(1):
#   for j in range(1):
#     data_loader(i,j);
    # train_and_evaluate(sess, 'dann_train_op', 'total_loss', verbose=False)




In [ ]:
# domains=['Kindle_Store_5','Books_5','Pet_Supplies_5','Home_and_Kitchen_5','Sports_and_Outdoors_5','Electronics_5']
# train_and_evaluate(sess, 'dann_train_op', 'total_loss', verbose=False)

for i in range(6):
  for j in range(6):
    data_loader(i,j);
    # print( domains[i] +" vs "+domains[j] )
    train_and_evaluate(sess, 'dann_train_op', 'total_loss', verbose=False,ix=i,iy=j)
    print(results)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
<ipython-input-18-94ed623c1990>:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized_train = {k:torch.tensor(v

Kindle_Store_5 vs Kindle_Store_5
Source domain:  0.4281184
Source class:  0.8562368
Target domain:  0.201373
Target class:  0.84324944
[[{'Source Domain': 0.4281184, 'Source Class': 0.8562368, 'Target Domain': 0.201373, 'Target Class': 0.84324944}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}]]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Kindle_Store_5 vs Books_5
Source domain:  0.41120508
Source class:  0.8435518
Target domain:  0.29290617
Target class:  0.8089245
[[{'Source Domain': 0.4281184, 'Source Class': 0.8562368, 'Target Domain': 0.201373, 'Target Class': 0.84324944}, {'Source Domain': 0.41120508, 'Source Class': 0.8435518, 'Target Domain': 0.29290617, 'Target Class': 0.8089245}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}], [{}, {}, {}, {}, {}, {}]]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Electronics_5 vs Home_and_Kitchen_5
Source domain:  0.41120508
Source class:  0.84143764
Target domain:  0.26887873
Target class:  0.82494277
[[{'Source Domain': 0.4281184, 'Source Class': 0.8562368, 'Target Domain': 0.201373, 'Target Class': 0.84324944}, {'Source Domain': 0.41120508, 'Source Class': 0.8435518, 'Target Domain': 0.29290617, 'Target Class': 0.8089245}, {'Source Domain': 0.6215645, 'Source Class': 0.832981, 'Target Domain': 0.18764302, 'Target Class': 0.79290617}, {'Source Domain': 0.4820296, 'Source Class': 0.82241017, 'Target Domain': 0.26773456, 'Target Class': 0.812357}, {'Source Domain': 0.41754758, 'Source Class': 0.8498943, 'Target Domain': 0.23455377, 'Target Class': 0.7894737}, {'Source Domain': 0.3245243, 'Source Class': 0.7928118, 'Target Domain': 0.30549198, 'Target Class': 0.791762}], [{'Source Domain': 0.38160676, 'Source Class': 0.8308668, 'Target Domain': 0.3249428, 'Target Class': 0.7768879}, {'Source Domain': 0.38266385, 'Source Class': 0.8625793, 'Targe

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Electronics_5 vs Sports_and_Outdoors_5
Source domain:  0.34038055
Source class:  0.8298097
Target domain:  0.31235698
Target class:  0.7871854
[[{'Source Domain': 0.4281184, 'Source Class': 0.8562368, 'Target Domain': 0.201373, 'Target Class': 0.84324944}, {'Source Domain': 0.41120508, 'Source Class': 0.8435518, 'Target Domain': 0.29290617, 'Target Class': 0.8089245}, {'Source Domain': 0.6215645, 'Source Class': 0.832981, 'Target Domain': 0.18764302, 'Target Class': 0.79290617}, {'Source Domain': 0.4820296, 'Source Class': 0.82241017, 'Target Domain': 0.26773456, 'Target Class': 0.812357}, {'Source Domain': 0.41754758, 'Source Class': 0.8498943, 'Target Domain': 0.23455377, 'Target Class': 0.7894737}, {'Source Domain': 0.3245243, 'Source Class': 0.7928118, 'Target Domain': 0.30549198, 'Target Class': 0.791762}], [{'Source Domain': 0.38160676, 'Source Class': 0.8308668, 'Target Domain': 0.3249428, 'Target Class': 0.7768879}, {'Source Domain': 0.38266385, 'Source Class': 0.8625793, 'Targ

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Electronics_5 vs Electronics_5
Source domain:  0.3742072
Source class:  0.82663846
Target domain:  0.26201373
Target class:  0.812357
[[{'Source Domain': 0.4281184, 'Source Class': 0.8562368, 'Target Domain': 0.201373, 'Target Class': 0.84324944}, {'Source Domain': 0.41120508, 'Source Class': 0.8435518, 'Target Domain': 0.29290617, 'Target Class': 0.8089245}, {'Source Domain': 0.6215645, 'Source Class': 0.832981, 'Target Domain': 0.18764302, 'Target Class': 0.79290617}, {'Source Domain': 0.4820296, 'Source Class': 0.82241017, 'Target Domain': 0.26773456, 'Target Class': 0.812357}, {'Source Domain': 0.41754758, 'Source Class': 0.8498943, 'Target Domain': 0.23455377, 'Target Class': 0.7894737}, {'Source Domain': 0.3245243, 'Source Class': 0.7928118, 'Target Domain': 0.30549198, 'Target Class': 0.791762}], [{'Source Domain': 0.38160676, 'Source Class': 0.8308668, 'Target Domain': 0.3249428, 'Target Class': 0.7768879}, {'Source Domain': 0.38266385, 'Source Class': 0.8625793, 'Target Domain